In [5]:
import pandas as pd
import json
import os
import re

def preprocess_cosmetic_data():
    """
    올리브영 제품 데이터와 성분 데이터베이스를 결합하여
    머신러닝 학습에 적합한 형태로 전처리하고 CSV 파일로 저장하는 함수입니다.
    """
    try:
        # --- 1. 데이터 로드 ---
        print("데이터 로딩을 시작합니다...")
        coos_df = pd.read_csv('coos_ingredient_database.csv')
        
        json_files = [
            'oliveyoung_로션_raw_limited.json',
            'oliveyoung_미스트_오일_raw_limited.json',
            'oliveyoung_스킨_토너_raw_limited.json',
            'oliveyoung_에센스_세럼_앰플_raw_limited.json',
            'oliveyoung_크림_raw_limited.json'
        ]

        all_products = []
        for file in json_files:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # 파일명에서 '카테고리' 정보 추출
                category = os.path.basename(file).replace('oliveyoung_', '').replace('_raw_limited.json', '')
                for item in data:
                    item['카테고리'] = category
                    all_products.append(item)
        
        oliveyoung_df = pd.DataFrame(all_products)
        print(f"총 {len(oliveyoung_df)}개의 제품 데이터를 로드했습니다.")

        # --- 2. 데이터 전처리 ---
        print("데이터 전처리를 시작합니다...")

        # 브랜드명, 제품명 분리 함수
        def extract_brand_and_clean_name(name):
            brand_match = re.match(r'\[([^\]]+)\]\s*(.*)', name)
            if brand_match:
                parts = brand_match.group(2).split()
                if parts:
                    return parts[0], ' '.join(parts[1:])
                else:
                    return brand_match.group(1), ""
            else:
                parts = name.split()
                if len(parts) > 1:
                    return parts[0], ' '.join(parts[1:])
                else:
                    return name, ""

        oliveyoung_df[['브랜드명', '제품명_정리']] = oliveyoung_df['제품명'].apply(
            lambda x: pd.Series(extract_brand_and_clean_name(x))
        )

        # 성분 데이터베이스 set 생성 (빠른 탐색용)
        master_ingredients_set = set(coos_df['원료명'].dropna().unique())

        # 성분 리스트 정제 함수
        def clean_and_normalize_ingredients(ingredient_list):
            cleaned_ingredients = set()
            pattern = re.compile(r'\s*\([^)]*\)|\[[^\]]*\]|\*|\'|"|\s+ppm|\s+ppb|\d+ml\+\d+ml', re.IGNORECASE)
            
            for ing_raw in ingredient_list:
                ing_cleaned = pattern.sub('', ing_raw).strip()
                
                if not ing_cleaned or ing_cleaned.isdigit():
                    continue

                if ing_cleaned in master_ingredients_set:
                    cleaned_ingredients.add(ing_cleaned)
            return list(cleaned_ingredients)

        oliveyoung_df['성분리스트_정리'] = oliveyoung_df['성분리스트'].apply(clean_and_normalize_ingredients)
        print("브랜드명, 제품명, 성분리스트 정제가 완료되었습니다.")

        # --- 3. Feature Matrix 생성 ---
        print("Feature Matrix 생성을 시작합니다...")
        all_unique_ingredients = sorted(list(master_ingredients_set))
        
        feature_df = pd.DataFrame(0, index=oliveyoung_df.index, columns=all_unique_ingredients)

        for index, row in oliveyoung_df.iterrows():
            for ingredient in row['성분리스트_정리']:
                if ingredient in feature_df.columns:
                    feature_df.loc[index, ingredient] = 1
        
        final_df = pd.concat([
            oliveyoung_df[['브랜드명', '제품명', '카테고리']],
            feature_df
        ], axis=1)
        print("Feature Matrix 생성이 완료되었습니다.")

        # --- 4. 결과 저장 ---
        output_filename = 'preprocessed_cosmetics_data.csv'
        final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\n✨ 모든 작업이 완료되었습니다! 결과가 '{output_filename}' 파일로 저장되었습니다.")
        print("\n[처리된 데이터 샘플]")
        print(final_df.head())

    except FileNotFoundError as e:
        print(f"오류: 파일을 찾을 수 없습니다. '{e.filename}' 파일이 코드와 동일한 폴더에 있는지 확인해주세요.")
    except Exception as e:
        print(f"데이터 처리 중 오류가 발생했습니다: {e}")

preprocess_cosmetic_data()

데이터 로딩을 시작합니다...
총 2187개의 제품 데이터를 로드했습니다.
데이터 전처리를 시작합니다...
브랜드명, 제품명, 성분리스트 정제가 완료되었습니다.
Feature Matrix 생성을 시작합니다...
Feature Matrix 생성이 완료되었습니다.

✨ 모든 작업이 완료되었습니다! 결과가 'preprocessed_cosmetics_data.csv' 파일로 저장되었습니다.

[처리된 데이터 샘플]
     브랜드명                                                제품명 카테고리   γ-터피넨  \
0  아이디얼포맨              [1등올인원] 아이디얼포맨 퍼펙트 올인원 기획세트(+100ml증정)   로션       0   
1  온그리디언츠   [화잘먹로션/속광케어] 온그리디언츠 스킨 베리어 카밍 로션 기획 (220ml+80ml)   로션       0   
2    피지오겔                피지오겔 DMT 페이셜 로션 200ml 기획 (+로션 50ml)   로션       0   
3    에스트라  에스트라 아토베리어365 로션 150ml 기획 (하이드로에센스 25ml+세라-히알 ...   로션       0   
4     닥터지  [리뉴얼/트러블/면도후] 닥터지 레드 블레미쉬 포 맨 진정 올인원 150ml 2종 ...   로션       0   

   (+/-)-테트라하이드로퍼푸릴-(R)-2-[4-(6-클로로퀴노살린-2-일옥시)페닐옥시]프로피오네이트  \
0                                                  0         
1                                                  0         
2                                                  0         
3                                                  0       

In [6]:
import pandas as pd
import json
import os
import re

def preprocess_to_string_efficiently():
    """
    올리브영 제품 데이터와 성분 데이터베이스를 결합하여,
    정제된 성분 리스트를 하나의 문자열로 만들어 CSV로 저장하는 효율적인 함수입니다.
    """
    try:
        # --- 1. 데이터 로드 ---
        print("데이터 로딩을 시작합니다...")
        coos_df = pd.read_csv('coos_ingredient_database.csv')
        
        json_files = [f for f in os.listdir('.') if f.startswith('oliveyoung_') and f.endswith('.json')]
        if not json_files:
            print("오류: 'oliveyoung_'으로 시작하는 JSON 파일을 찾을 수 없습니다.")
            return

        all_products = []
        for file in json_files:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                category = os.path.basename(file).replace('oliveyoung_', '').replace('_raw_limited.json', '')
                for item in data:
                    item['카테고리'] = category
                    all_products.append(item)
        
        oliveyoung_df = pd.DataFrame(all_products)
        print(f"총 {len(oliveyoung_df)}개의 제품 데이터를 로드했습니다.")

        # --- 2. 브랜드명 및 제품명 정제 ---
        print("브랜드명과 제품명 정제를 시작합니다...")

        def extract_brand_and_clean_name(name):
            if not isinstance(name, str): return "", ""
            brand_match = re.match(r'\[([^\]]+)\]\s*(.*)', name)
            if brand_match:
                parts = brand_match.group(2).split()
                return (parts[0], ' '.join(parts[1:])) if parts else (brand_match.group(1), "")
            else:
                parts = name.split()
                return (parts[0], ' '.join(parts[1:])) if len(parts) > 1 else (name, "")

        oliveyoung_df[['브랜드명', '제품명_정리']] = oliveyoung_df['제품명'].apply(
            lambda x: pd.Series(extract_brand_and_clean_name(x))
        )

        # --- 3. 성분 리스트 정제 ---
        print("성분 리스트 정제를 시작합니다...")
        master_ingredients_set = set(coos_df['원료명'].dropna().unique())

        def clean_and_normalize_ingredients(ingredient_list):
            if not isinstance(ingredient_list, list): return []
            cleaned_ingredients = set()
            for ing_raw in ingredient_list:
                if not isinstance(ing_raw, str): continue
                
                temp_ing = re.sub(r'\[[^\]]+\]|\([^)]+\)|\*|"|\'|\s+ppm|\s+ppb|\d+ml\+\d+ml', '', ing_raw, flags=re.IGNORECASE)
                tokens = temp_ing.split()
                
                for token in tokens:
                    token = token.strip()
                    if token and not token.isdigit() and token in master_ingredients_set:
                        cleaned_ingredients.add(token)
            return sorted(list(cleaned_ingredients)) # 일관된 순서를 위해 정렬

        oliveyoung_df['성분리스트_정리'] = oliveyoung_df['성분리스트'].apply(clean_and_normalize_ingredients)
        print("성분 리스트 정제가 완료되었습니다.")

        # --- 4. 정제된 성분을 하나의 문자열로 합치기 ---
        print("정제된 성분들을 하나의 문자열로 합치는 중입니다...")
        oliveyoung_df['성분_문자열'] = oliveyoung_df['성분리스트_정리'].apply(lambda x: ' '.join(x))

        # --- 5. 최종 데이터프레임 생성 및 저장 ---
        final_df = oliveyoung_df[['브랜드명', '제품명', '카테고리', '성분_문자열']]
        
        output_filename = 'processed_cosmetics_string.csv'
        final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        
        print(f"\n✨ 모든 작업이 완료되었습니다! 결과가 '{output_filename}' 파일로 저장되었습니다.")
        print("\n[처리된 데이터 샘플]")
        print(final_df.head())

    except FileNotFoundError as e:
        print(f"\n오류: 파일을 찾을 수 없습니다. '{e.filename}' 파일이 코드와 동일한 폴더에 있는지 확인해주세요.")
    except Exception as e:
        print(f"\n데이터 처리 중 오류가 발생했습니다: {e}")

preprocess_to_string_efficiently()

데이터 로딩을 시작합니다...
총 2287개의 제품 데이터를 로드했습니다.
브랜드명과 제품명 정제를 시작합니다...
성분 리스트 정제를 시작합니다...
성분 리스트 정제가 완료되었습니다.
정제된 성분들을 하나의 문자열로 합치는 중입니다...

✨ 모든 작업이 완료되었습니다! 결과가 'processed_cosmetics_string.csv' 파일로 저장되었습니다.

[처리된 데이터 샘플]
   브랜드명                                                제품명       카테고리  \
0   웰라쥬  [10월 올영픽/150ml한정기획+마스크] 웰라쥬 리얼 히알루로닉 블루 100 앰플...  에센스_세럼_앰플   
1   메디힐       [단독/1+1] 메디힐 마데카소사이드 흔적 리페어 세럼 40+40ml 더블 기획  에센스_세럼_앰플   
2  브링그린                      브링그린 징크테카 트러블 세럼 (대용량/콜라보/기획)  에센스_세럼_앰플   
3    구달   [리뉴얼] 구달 청귤 비타C 잡티케어 세럼 알파 50ml 리필 기획 (+크림 10ml)  에센스_세럼_앰플   
4  메디큐브  [10월 올영픽/1등 미백앰플] 메디큐브 PDRN 핑크 앰플 30ml 리필기획 (본...  에센스_세럼_앰플   

                                              성분_문자열  
0  갈랑가갈랑갈추출물 글라이신 글라이코실트레할로오스 글리세레스-26 글리세린 글리세릴아...  
1  광곽향오일 글리세린 글리세릴폴리메타크릴레이트 나이아신아마이드 다마스크장미꽃수 다이페...  
2  4-터피네올 글라이코리피드 나이아신아마이드 다이소듐이디티에이 메틸글루세스-20 메틸...  
3  3-O-에틸아스코빅애씨드 감초뿌리추출물 글루타티온 글리세레스-26 글리세린 나이아신...  
4  글리세레스-26 글리세린 글리세릴아크릴레이트/아크릴릭애씨드코폴리머 나이아신아마이드 ...  


In [7]:
import pandas as pd
import json
import os
import re

def final_advanced_preprocessing_with_custom_keywords():
    """
    사용자가 제공한 샘플 데이터를 기반으로 추출한 키워드를 사용하여
    제품명 정제 로직을 고도화하고, 최종 CSV로 저장하는 함수입니다.
    """
    try:
        # --- 1. 데이터 로드 ---
        print("데이터 로딩을 시작합니다...")
        coos_df = pd.read_csv('coos_ingredient_database.csv')

        json_files = [f for f in os.listdir('.') if f.startswith('oliveyoung_') and f.endswith('.json')]
        if not json_files:
            print("오류: 'oliveyoung_'으로 시작하는 JSON 파일을 찾을 수 없습니다.")
            return

        all_products = []
        for file in json_files:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                category = os.path.basename(file).replace('oliveyoung_', '').replace('_raw_limited.json', '')
                for item in data:
                    item['카테고리'] = category
                    all_products.append(item)

        oliveyoung_df = pd.DataFrame(all_products)
        print(f"총 {len(oliveyoung_df)}개의 제품 데이터를 로드했습니다.")

        # --- 2. 제품명 및 브랜드명 정제 (사용자 데이터 기반 고도화) ---
        print("사용자 샘플 기반의 고도화된 로직으로 제품명과 브랜드명 정제를 시작합니다...")

        def extract_and_clean_product_info(original_name):
            if not isinstance(original_name, str):
                return "", "", ""

            # 브랜드명 후보 추출
            temp_name = original_name
            brand_match = re.match(r'\[[^\]]+\]\s*([^\s]+)', temp_name)
            if brand_match:
                brand_cleaned = brand_match.group(1)
            else:
                brand_cleaned = temp_name.split()[0] if temp_name.split() else ""

            # 제품명 정제 시작
            cleaned_name = re.sub(r'\[[^\]]+\]', '', temp_name)
            cleaned_name = re.sub(r'\([^)]+\)', '', cleaned_name)
            cleaned_name = re.sub(r'\s*\+.*', '', cleaned_name)

            cleaned_name = cleaned_name.replace(brand_cleaned, '', 1).strip()

            cleaned_name = re.sub(r'\b\d+(\.\d+)?\s*(ml|g|L|p|P)\b', '', cleaned_name, flags=re.I)
            cleaned_name = re.sub(r'\b\d+\s*[입매개eaEA]+\b', '', cleaned_name, flags=re.I)
            cleaned_name = re.sub(r'\b(\d+\+\d+|\d\+\d)\b', '', cleaned_name)

            promo_keywords = [
                '기획', '한정', '증정', '더블', '대용량', '리필', '올영픽', '단독', '콜라보', '세트', '리뉴얼',
                '알파', 'PICK', 'UP', '특가', '에디션', '아따맘마', '춘구마', '가나디', '포켓몬', '1등',
                '화잘먹', '흔적', '진정', '모공', '탄력', '수분', '미백', '잡티', '광채'
            ]
            for keyword in promo_keywords:
                cleaned_name = re.sub(r'\b' + re.escape(keyword) + r'\b', '', cleaned_name)

            cleaned_name = re.sub(r'\b\d+\b', '', cleaned_name)
            cleaned_name = ' '.join(cleaned_name.split()).strip()

            if not cleaned_name:
                cleaned_name = ' '.join(original_name.replace(brand_cleaned, '').split()).strip()

            return original_name, brand_cleaned, cleaned_name

        oliveyoung_df[['제품명_원본', '브랜드명_정리', '제품명_정리']] = oliveyoung_df['제품명'].apply(
            lambda x: pd.Series(extract_and_clean_product_info(x))
        )
        print("제품명 정제가 완료되었습니다.")

        # --- 3. 성분 리스트 정제 및 문자열 변환 ---
        print("성분 리스트 정제 및 문자열 변환을 시작합니다...")
        master_ingredients_set = set(coos_df['원료명'].dropna().unique())

        def clean_and_stringify_ingredients(ingredient_list):
            if not isinstance(ingredient_list, list): return ""
            cleaned_ingredients = set()
            for ing_raw in ingredient_list:
                if not isinstance(ing_raw, str): continue
                temp_ing = re.sub(r'\[[^\]]+\]|\([^)]+\)|\*|"|\'|\s+ppm|\s+ppb|\d+ml\+\d+ml', '', ing_raw, flags=re.I)
                tokens = temp_ing.split()
                for token in tokens:
                    token = token.strip()
                    if token and not token.isdigit() and token in master_ingredients_set:
                        cleaned_ingredients.add(token)
            return ' '.join(sorted(list(cleaned_ingredients)))

        oliveyoung_df['성분_문자열'] = oliveyoung_df['성분리스트'].apply(clean_and_stringify_ingredients)
        print("성분 데이터 처리가 완료되었습니다.")

        # --- 4. 최종 데이터프레임 생성 및 저장 ---
        final_df = oliveyoung_df[[
            '브랜드명_정리',
            '제품명_정리',
            '카테고리',
            '성분_문자열',
            '제품명_원본'
        ]]

        output_filename = 'processed_cosmetics_final.csv'
        final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

        print(f"\n✨ 모든 작업이 완료되었습니다! 결과가 '{output_filename}' 파일로 저장되었습니다.")
        print("\n[처리된 데이터 샘플]")
        print(final_df.head())

    except FileNotFoundError as e:
        print(f"\n오류: 파일을 찾을 수 없습니다. '{e.filename}' 파일이 코드와 동일한 폴더에 있는지 확인해주세요.")
    except Exception as e:
        print(f"\n데이터 처리 중 오류가 발생했습니다: {e}")

final_advanced_preprocessing_with_custom_keywords()

데이터 로딩을 시작합니다...
총 2287개의 제품 데이터를 로드했습니다.
사용자 샘플 기반의 고도화된 로직으로 제품명과 브랜드명 정제를 시작합니다...
제품명 정제가 완료되었습니다.
성분 리스트 정제 및 문자열 변환을 시작합니다...
성분 데이터 처리가 완료되었습니다.

✨ 모든 작업이 완료되었습니다! 결과가 'processed_cosmetics_final.csv' 파일로 저장되었습니다.

[처리된 데이터 샘플]
  브랜드명_정리           제품명_정리       카테고리  \
0     웰라쥬   리얼 히알루로닉 블루 앰플  에센스_세럼_앰플   
1     메디힐   마데카소사이드 리페어 세럼  에센스_세럼_앰플   
2    브링그린      징크테카 트러블 세럼  에센스_세럼_앰플   
3      구달   청귤 비타C 잡티케어 세럼  에센스_세럼_앰플   
4    메디큐브  PDRN 핑크 앰플 리필기획  에센스_세럼_앰플   

                                              성분_문자열  \
0  갈랑가갈랑갈추출물 글라이신 글라이코실트레할로오스 글리세레스-26 글리세린 글리세릴아...   
1  광곽향오일 글리세린 글리세릴폴리메타크릴레이트 나이아신아마이드 다마스크장미꽃수 다이페...   
2  4-터피네올 글라이코리피드 나이아신아마이드 다이소듐이디티에이 메틸글루세스-20 메틸...   
3  3-O-에틸아스코빅애씨드 감초뿌리추출물 글루타티온 글리세레스-26 글리세린 나이아신...   
4  글리세레스-26 글리세린 글리세릴아크릴레이트/아크릴릭애씨드코폴리머 나이아신아마이드 ...   

                                              제품명_원본  
0  [10월 올영픽/150ml한정기획+마스크] 웰라쥬 리얼 히알루로닉 블루 100 앰플...  
1       [단독/1+1] 메디힐 마데카소사이드 흔적 리페어 세럼 40+40ml 더블 기획  
2                